<a href="https://colab.research.google.com/github/mavillan/indoor-location-navigation/blob/main/train/iln_dnn_ds1_40f_5lt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install --upgrade kaggle > /dev/null 2>&1
!pip install pytorch_axe > /dev/null 2>&1
!pip install adabelief-pytorch > /dev/null 2>&1

In [ ]:
# move kaggle.json into the folder where the API expects to find it
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

***

In [ ]:
import gc
import os
import numpy as np
import pandas as pd
from glob import glob
from sklearn import preprocessing
import yaml

import torch
from torch import nn
from torch.utils.data import DataLoader,TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from adabelief_pytorch import AdaBelief
import pytorch_axe as pax
import time

# device in which the model will be trained
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def mean_position_error(x_label, y_label, x_pred, y_pred):
    return np.mean(np.sqrt((x_label-x_pred)**2 + (y_label-y_pred)**2))

@torch.jit.script
def mpe_loss(preds, targets):
    pw_loss = torch.sqrt((preds[:,0]-targets[:,0])**2 + (preds[:,1]-targets[:,1])**2)
    return torch.mean(pw_loss)

In [ ]:
def train_encoder(train_frame, test_frame, bssid_cols):
    bssid_unique_train = set(np.unique(train_frame[bssid_cols].values))
    bssid_unique_test = set(np.unique(test_frame[bssid_cols].values))
    bssid_unique = np.array(list(bssid_unique_train | bssid_unique_test))
    encoder = preprocessing.LabelEncoder()
    encoder.fit(bssid_unique)
    return encoder

def encode_bssids(dataframe, bssid_cols, encoder):
    for col in bssid_cols:
        dataframe[col] = encoder.transform(dataframe[col].values)
    return dataframe

In [ ]:
# stratified group kfold split - stratified by flood and grouped by path

def cv_split(dataframe, n_splits, random_state=2):
    np.random.seed(random_state)

    fold_indexes = [list() for i in range(n_splits)]

    for floor in np.sort(dataframe.floor.unique()):
        _dataframe = dataframe[dataframe["floor"]==floor]

        # split of paths
        unique_paths = _dataframe.path.unique()
        np.random.shuffle(unique_paths)
        paths_per_fold = np.array_split(unique_paths, n_splits)

        for i,paths in enumerate(paths_per_fold):
            fold_indexes[i].append(_dataframe.query("path in @paths").index.values)

    fold_indexes = [np.concatenate(indexes) for indexes in fold_indexes]
    
    for i in range(n_splits):
        train_idx = np.concatenate([idx for j,idx in enumerate(fold_indexes) if j!=i])
        valid_idx = fold_indexes[i]
        yield train_idx,valid_idx

In [ ]:
project_path = "/content/drive/MyDrive/kaggle/indoor-location-navigation"

***

In [ ]:
# params to configure the dataset
MAX_WIFI_FEATS = 40
MAX_LASTSEEN = 5
MAX_TIME_DIFF = 10000

# params for reproducibility
N_SPLITS = 5
SEED = 2
CV_SEED = 2
set_seed(SEED)

# params of model
BATCH_SIZE = 2048
EMBEDDING_DIM = 12

In [ ]:
!kaggle datasets download -d $(echo "mavillan/iln-ds1-{MAX_WIFI_FEATS}f-{MAX_LASTSEEN}lt") --force --unzip

 95% 266M/281M [00:04<00:00, 35.2MB/s]
100% 281M/281M [00:04<00:00, 60.9MB/s]


***

In [ ]:
train_files = sorted(glob(f"./train/*.parquet"))
test_files = sorted(glob(f"./test/*.parquet"))
sites = [f.split("/")[-1].split(".")[0] for f in train_files]

In [ ]:
floor_predictions = pd.read_csv(f"{project_path}/input/simple-99-accurate-floor-model/submission.csv")
floor_predictions.drop(["x","y"], axis=1, inplace=True)

In [ ]:
# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {
    "B2":-2, "B1":-1, 
    "F1":0, "F2":1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6, "F8":7, "F9":8,
    "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F":7, "9F":8
}

In [ ]:
# features & target definition
bssid_cols = [f"bssid_{i}" for i in range(MAX_WIFI_FEATS)]
rssi_cols = [f"rssi_{i}" for i in range(MAX_WIFI_FEATS)]
delta_cols = [f"deltaseen_{i}" for i in range(MAX_WIFI_FEATS)]
wifi_cols = bssid_cols + rssi_cols + delta_cols
            
nowifi_cols = [
    "floor", 
    "imu_time_delta", "wifi_time_delta",
    "acce_x", "acce_y", "acce_z", 
    #"acce_uncali_x", "acce_uncali_y", "acce_uncali_z",
    "gyro_x", "gyro_y", "gyro_z", 
    #"gyro_uncali_x", "gyro_uncali_y", "gyro_uncali_z",
    "magn_x", "magn_y", "magn_z",
    #"magn_uncali_x", "magn_uncali_y", "magn_uncali_z",
    "ahrs_x", "ahrs_y", "ahrs_z",   
]

target_cols = ["x","y"]

***
## model definition

In [ ]:
class DNN(nn.Module):

    def __init__(self, input_dims, output_dim, nn_depth, nn_width, dropout, momentum,
                 n_embeddings, embedding_dim):
        super().__init__()

        self.input_dims = input_dims
        self.n_wifi_feats = input_dims[0]
        self.embedder = nn.Embedding(n_embeddings, embedding_dim, max_norm=True)
        
        self.bnorms_parallel = nn.ModuleList(
            [nn.BatchNorm1d(embedding_dim+2, momentum=momentum) 
            for i in range(self.n_wifi_feats)]
            )
        self.dropouts_parallel = nn.ModuleList(
            [nn.Dropout(dropout/2) 
            for i in range(self.n_wifi_feats)]
            )
        self.linears_parallel = nn.ModuleList(
            [nn.utils.weight_norm(nn.Linear(embedding_dim+2, embedding_dim, bias=False))
            for i in range(self.n_wifi_feats)]
            )
        
        input_dim = self.n_wifi_feats*embedding_dim + input_dims[3]
        self.bn_in = nn.BatchNorm1d(input_dim, momentum=momentum)
        self.dp_in = nn.Dropout(dropout/2)
        self.ln_in = nn.utils.weight_norm(nn.Linear(input_dim, nn_width, bias=False))

        self.bnorms = nn.ModuleList([
            nn.BatchNorm1d(nn_width, momentum=momentum) 
            for i in range(nn_depth-1)])
        self.dropouts = nn.ModuleList([
            nn.Dropout(dropout) 
            for i in range(nn_depth-1)])
        self.linears = nn.ModuleList([
            nn.utils.weight_norm(nn.Linear(nn_width, nn_width, bias=False))
            for i in range(nn_depth-1)])
        
        self.bn_out = nn.BatchNorm1d(nn_width, momentum=momentum)
        self.dp_out = nn.Dropout(dropout/2)
        self.ln_out = nn.utils.weight_norm(nn.Linear(nn_width, output_dim, bias=False))

    def forward(self, x_bssid, x_rssi, x_delta, x_rest):
        
        # processing of bssid & rssi features
        x_encoded = self.embedder(x_bssid)
        all_outs = list()
        for i in range(self.n_wifi_feats):
            x_tmp = torch.cat([x_encoded[:,i,:],  x_rssi[:,i].reshape(-1,1), x_delta[:,i].reshape(-1,1)], dim=1)
            x_tmp = self.bnorms_parallel[i](x_tmp)
            x_tmp = self.dropouts_parallel[i](x_tmp)
            x_tmp = nn.functional.relu(self.linears_parallel[i](x_tmp))
            all_outs.append(x_tmp)
            
        all_outs.append(x_rest)
        x = torch.cat(all_outs, dim=1)
        
        x = self.bn_in(x)
        x = self.dp_in(x)
        x = nn.functional.relu(self.ln_in(x))

        for bn_layer,dp_layer,ln_layer in zip(self.bnorms,self.dropouts,self.linears):
            x = bn_layer(x)
            x = dp_layer(x)
            x = ln_layer(x)
            x = nn.functional.relu(x)
            
        x = self.bn_out(x)
        x = self.dp_out(x)
        x = self.ln_out(x)
        return x

    def training_step(self, batch):
        x_bssid, x_rssi, x_delta, x_rest, y = batch
        y_hat = self.forward(x_bssid, x_rssi, x_delta, x_rest)
        loss = mpe_loss(y_hat, y)
        return loss
    
    def validation_step(self, batch):
        x_bssid, x_rssi, x_delta, x_rest, y = batch
        y_hat = self.forward(x_bssid, x_rssi, x_delta, x_rest)
        loss = mpe_loss(y_hat, y)
        return loss
    
    def prediction_step(self, batch):
        x_bssid, x_rssi, x_delta, x_rest, _ = batch
        pred = self.forward(x_bssid, x_rssi, x_delta, x_rest)
        return pred
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5, min_lr=1e-5)
        return optimizer,scheduler

***
## model training by site

In [ ]:
%%time

oofs_by_site = list()
preds_by_site = list()
preds_by_site_raw = list()
cv_errors = dict()

for i,(train_file,test_file,site) in enumerate(zip(train_files,test_files,sites)):
    print(f" training models for site {i+1}/24: {site} ".center(70, "#"))
    
    train = pd.read_parquet(train_file)
    test = pd.read_parquet(test_file)
    
    train["site"] = site
    test["site"] = site

    # adds floor predictions on test
    train["floor"] = train.floor.map(floor_map)
    test = pd.merge(test, floor_predictions, how="left", on="site_path_timestamp")

    # filter wifi records that are near (in time) to waypoints
    train = train.query("wifi_time_diff <= @MAX_TIME_DIFF | wifi_idx==0").reset_index(drop=True)
    test = test.query("wifi_time_diff <= @MAX_TIME_DIFF | wifi_idx==0").reset_index(drop=True)
    
    train_bssids = set(np.unique(train[bssid_cols].values))
    test_bssids = set(np.unique(test[bssid_cols].values))
    print(f"bssids diff:", test_bssids - train_bssids)
        
    # integer encoder for string features
    encoder = train_encoder(train, test, bssid_cols)
    n_bssids = len(encoder.classes_)
    print("number of bssids:", n_bssids)
    
    # cv split here
    mskf = cv_split(train, n_splits=N_SPLITS, random_state=CV_SEED)
    all_models = list()
    
    oof_cols = ["site","path","timestamp","wifi_time_delta","wifi_time_diff","seq_nbr","x","y"]
    oof_preds = train.loc[:,oof_cols].copy()
    oof_preds["oof_idx"] = -1
    oof_preds["oof_x"] = np.nan
    oof_preds["oof_y"] = np.nan
    
    for i,(train_idx,valid_idx) in enumerate(mskf):
        print(f"training for fold {i+1}/{N_SPLITS}".center(70, "-"))
        
        _train = train.loc[train_idx, wifi_cols+nowifi_cols+target_cols]
        _valid = train.loc[valid_idx, wifi_cols+nowifi_cols+target_cols]
        
        _train = encode_bssids(_train, bssid_cols, encoder)
        _valid = encode_bssids(_valid, bssid_cols, encoder)
        
        train_dset = TensorDataset(
            torch.tensor(_train[bssid_cols].values, dtype=torch.long),
            torch.tensor(_train[rssi_cols].values, dtype=torch.float),
            torch.tensor(_train[delta_cols].values, dtype=torch.float),
            torch.tensor(_train[nowifi_cols].values, dtype=torch.float),
            torch.tensor(_train[target_cols].values, dtype=torch.float)
        )

        valid_dset = TensorDataset(
            torch.tensor(_valid[bssid_cols].values, dtype=torch.long),
            torch.tensor(_valid[rssi_cols].values, dtype=torch.float),
            torch.tensor(_valid[delta_cols].values, dtype=torch.float),
            torch.tensor(_valid[nowifi_cols].values, dtype=torch.float),
            torch.tensor(_valid[target_cols].values, dtype=torch.float)
        )
        
        input_dims = (len(bssid_cols),len(rssi_cols),len(delta_cols),len(nowifi_cols))
        model = DNN(
            input_dims=input_dims,
            output_dim=2, 
            nn_depth=3, 
            nn_width=256, 
            dropout=0.2, 
            momentum=0.1,
            n_embeddings=n_bssids,
            embedding_dim=EMBEDDING_DIM,
        )
        
        train_dataloader = DataLoader(train_dset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=len(valid_dset), shuffle=False, num_workers=2, pin_memory=True)
        
        model,monitor = pax.iterative_train(model, train_dataloader, valid_dataloader, max_epochs=200, patience=20)
        model.load_state_dict(monitor.best_model_state)
        
        # saving oof predictions
        preds = pax.iterative_predict(model, valid_dataloader).detach().cpu().numpy()
        oof_preds.loc[valid_idx, "oof_idx"] = i
        oof_preds.loc[valid_idx, "oof_x"] = preds[:,0]
        oof_preds.loc[valid_idx, "oof_y"] = preds[:,1]
        
        all_models.append(model)
                
        del train_dataloader,valid_dataloader
        del train_dset,valid_dset
        gc.collect()
    
    # calculating the oof errors
    oof_preds["oof_x_agg"] = oof_preds.groupby(["path","seq_nbr"])["oof_x"].transform("mean")
    oof_preds["oof_y_agg"] = oof_preds.groupby(["path","seq_nbr"])["oof_y"].transform("mean")  
    oof_preds_agg = oof_preds.loc[:,["x","y","oof_x_agg","oof_y_agg"]].drop_duplicates()
    mpe1 = mean_position_error(oof_preds.x, oof_preds.y, oof_preds.oof_x, oof_preds.oof_y)
    mpe2 = mean_position_error(oof_preds_agg.x, oof_preds_agg.y, oof_preds_agg.oof_x_agg, oof_preds_agg.oof_y_agg)
    cv_errors[site] = {"mpe1":mpe1, "mpe2":mpe2,}
    print(f"site: {site} - mpe1: {mpe1} - mpe2: {mpe2} \n")
         
    # generating the predictions for test dataset
    test = encode_bssids(test, bssid_cols, encoder)
    test_bssid = torch.tensor(test[bssid_cols].values, dtype=torch.long, device=device)
    test_rssi = torch.tensor(test[rssi_cols].values, dtype=torch.float, device=device)
    test_delta = torch.tensor(test[delta_cols].values, dtype=torch.float, device=device)
    test_nowifi = torch.tensor(test[nowifi_cols].values, dtype=torch.float, device=device)

    all_preds = list()
    for model in all_models:
        model.eval()
        with torch.set_grad_enabled(False):
            preds = model(test_bssid, test_rssi, test_delta, test_nowifi).detach().cpu().numpy()
            all_preds.append(preds)
        
    preds_avg = np.mean(all_preds, axis=0)
    test["x_pred"] = preds_avg[:,0]
    test["y_pred"] = preds_avg[:,1]
    test["x"] = test.groupby(["path","seq_nbr"])["x_pred"].transform("mean")
    test["y"] = test.groupby(["path","seq_nbr"])["y_pred"].transform("mean")
    
    oofs_by_site.append(oof_preds)
    preds_by_site.append(test.loc[:, ["site_path_timestamp","floor","x","y"]].drop_duplicates())
    preds_by_site_raw.append(test.loc[:,["site_path_timestamp","site","path","timestamp","wifi_time_delta","wifi_time_diff","seq_nbr","x","y","x_pred","y_pred"]])


###### training models for site 1/24: 5a0546857ecc773753327266 #######
bssids diff: set()
number of bssids: 2710
------------------------training for fold 1/5-------------------------



------------------------training for fold 2/5-------------------------



------------------------training for fold 3/5-------------------------



------------------------training for fold 4/5-------------------------



------------------------training for fold 5/5-------------------------


...


------------------------training for fold 2/5-------------------------



------------------------training for fold 3/5-------------------------



------------------------training for fold 4/5-------------------------



------------------------training for fold 5/5-------------------------



site: 5dc8cea7659e181adb076a3f - mpe1: 8.552760588834452 - mpe2: 6.977390350337759 

CPU times: user 2h 11min 19s, sys: 49min 30s, total: 3h 49s
Wall time: 8h 21min 52s


In [ ]:
output_path = f"{project_path}/output/dnn-ds1-{MAX_WIFI_FEATS}f-{MAX_LASTSEEN}lt"
if not os.path.exists(output_path): os.makedirs(output_path)

# save the oof preds
oof_preds = pd.concat(oofs_by_site, ignore_index=True)
oof_preds.to_csv(f"{output_path}/oof_preds.csv", index=False)

# save raw test preds
submission_raw = pd.concat(preds_by_site_raw, ignore_index=True)
submission_raw.to_csv(f"{output_path}/submission_raw.csv", index=False)

# save the cv metrics
oof_preds_agg = oof_preds.loc[:,["x","y","oof_x_agg","oof_y_agg"]].drop_duplicates()
mpe1 = mean_position_error(
    oof_preds.x, oof_preds.y, 
    oof_preds.oof_x, oof_preds.oof_y)
mpe2 = mean_position_error(
    oof_preds_agg.x, oof_preds_agg.y, 
    oof_preds_agg.oof_x_agg, oof_preds_agg.oof_y_agg)
cv_errors["overall"] = {"mpe1":mpe1, "mpe2":mpe2,}
print({"mpe1":mpe1, "mpe2":mpe2,})

with open(f'{output_path}/cv_errors.yml', 'w') as file:
    yaml.dump(cv_errors, file, default_flow_style=False)
    file.close()

# save the final submission
sample_sub = pd.read_csv(f"{project_path}/input/sample_submission.csv")
predictions = pd.concat(preds_by_site, ignore_index=True)
submission = pd.merge(sample_sub.loc[:,["site_path_timestamp"]], predictions, how="left")
assert (sample_sub.site_path_timestamp == submission.site_path_timestamp).all()
submission.to_csv(f"{output_path}/submission.csv", index=False)

{'mpe1': 8.206883642202056, 'mpe2': 6.577376961285939}


- Current best: {'mpe1': 8.132519384855483, 'mpe2': 6.742418387295461}
- Description: "[colab] dnn-ds1-40f with weight_norm, batch_size=2048 & learning_rate=1e-2"
- Wall time: 6h 6min 53s

***